### Notebook for inspecting denoised protein expression with `Scanpy` with output from `totalVI`

#### Environment: Scanpy

- **Developed by:** Alexandra Cirnu
- **Modified by:** Alexandra Cirnu
- **Würzburg Institute for Systems Immunology & Julius-Maximilian-Universität Würzburg**
- **Date of creation:** 240601
- **Date of modification:** 240601

### Import required modules

In [ ]:
import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import muon as mu
from muon import atac as ac
from muon import prot as pt
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import rcParams
from scipy.sparse import csr_matrix

### Set up working environment

In [ ]:
%matplotlib inline
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 160, color_map = 'RdPu', dpi_save = 180, vector_friendly = True, format = 'svg')

In [ ]:
def X_is_raw(adata): return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

In [ ]:
def clr_normalize_each_cell(adata, inplace=True):
    """Normalize count vector for each cell, i.e. for each row of .X"""

    import numpy as np
    import scipy

    def seurat_clr(x):
        # TODO: support sparseness
        s = np.sum(np.log1p(x[x > 0]))
        exp = np.exp(s / len(x))
        return np.log1p(x / exp)

    if not inplace:
        adata = adata.copy()

    # apply to dense or sparse matrix, along axis. returns dense matrix
    adata.X = np.apply_along_axis(
        seurat_clr, 1, (adata.X.A if scipy.sparse.issparse(adata.X) else adata.X)
    )
    return adata

### Read in MuData set

In [ ]:
mdata = mu.read_h5mu('/Users/alex/data/ACM_cardiac_leuco/CITE-Seq/ACM_myeloids+lymphoids_integrated_CITE_surface_markers_ac240510.raw.h5mu')
mdata

In [ ]:
rna = mdata.mod['rna']
protein = mdata.mod['prot']

In [ ]:
protein_raw = protein.X.copy()

In [ ]:
protein.var.head(20)

In [ ]:
markers_to_remove = ['prot:Hashtag1_TotalA', 'prot:Hashtag2_TotalA', 'prot:Hashtag3_TotalA','prot:Hashtag4_TotalA', 'prot:Hashtag5_TotalA', 'prot:Hashtag6_TotalA','prot:Hashtag7_TotalA', 'prot:Hashtag8_TotalA', 'prot:Hashtag9_TotalA','prot:Hashtag10_TotalA']  # Replace these with actual gene names you want to remove

# Filter out these genes
protein = protein[:, ~protein.var.index.isin(markers_to_remove)]
protein

In [ ]:
protein.X = protein.layers['counts']

In [ ]:
X_data = protein.X.copy()
X_data_sparse = csr_matrix(X_data)
X_data_df = pd.DataFrame.sparse.from_spmatrix(X_data_sparse, index=protein.obs.index, columns=protein.var.index)
print("Shape of counts DataFrame:", X_data_df.shape)
X_data_df

In [ ]:
protein.X = protein.layers['denoised_protein']

In [ ]:
X_data = protein.X.copy()
X_data_sparse = csr_matrix(X_data)
X_data_df = pd.DataFrame.sparse.from_spmatrix(X_data_sparse, index=protein.obs.index, columns=protein.var.index)
print("Shape of counts DataFrame:", X_data_df.shape)
X_data_df

### Normalize denoised protein expression

In [ ]:
clr_normalize_each_cell(protein)

In [ ]:
X_data = protein.X.copy()
X_data_sparse = csr_matrix(X_data)
X_data_df = pd.DataFrame.sparse.from_spmatrix(X_data_sparse, index=protein.obs.index, columns=protein.var.index)
print("Shape of counts DataFrame:", X_data_df.shape)
X_data_df

### Perform PCA

In [ ]:
sc.tl.pca(protein, n_comps= 50)

### Find Neighbors

In [ ]:
sc.pp.neighbors(protein, n_neighbors=150)

### Compute UMAP

In [ ]:
sc.tl.umap(protein)

In [ ]:
sc.pl.umap(protein, title='UMAP of Protein Data', frameon= False)

### Cluster the cells

In [ ]:
sc.tl.leiden(protein)

In [ ]:
sc.pl.umap(protein, color=['leiden', 'C_scANVI', 'classification'], frameon = False, ncols=1)

### Differential expression analysis

In [ ]:
sc.tl.rank_genes_groups(protein, "leiden", method="wilcoxon", n_genes= 100)
sc.pl.rank_genes_groups(protein, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(protein.uns["rank_genes_groups"]["names"]).head(3)

In [ ]:
# Initialize an empty list to store arrays for vertical stacking
out = []
result = protein.uns["rank_genes_groups"]
groups = result["names"].dtype.names

# Iterate over each group
for group in groups:
    # Stack arrays vertically and append to the 'out' list
    out.append(np.vstack((
        result['names'][group],
        result['scores'][group],
        result['pvals_adj'][group],
        result['logfoldchanges'][group],
        np.array([group] * len(result['names'][group])).astype('object')
    )).T)

# Vertically stack all arrays in the 'out' list
out = np.vstack(out)

#Create a data frame
markers = pd.DataFrame(out, columns= ['Gene', 'Score', 'Adj_pvalue', 'Logfoldchange', 'Cluster'])
markers_filtered = markers[(markers.Adj_pvalue < 0.05) & (abs(markers.Logfoldchange) > 1)]
markers_filtered.shape

In [ ]:
sc.pl.rank_genes_groups_dotplot(protein, groupby= 'leiden', n_genes=2, cmap='RdPu')

In [ ]:
sc.pl.rank_genes_groups_dotplot(protein, groupby= 'classification', n_genes=3, cmap='RdPu')

In [ ]:
sc.pl.rank_genes_groups_dotplot(protein, groupby= 'C_scANVI', n_genes=3, cmap='RdPu')

In [ ]:
num_genes = len(protein.var_names)
ncols = 8
nrows = (num_genes + ncols - 1) // ncols  # Calculate rows needed

# Create a large figure to hold all subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5*ncols, 5*nrows))
axes = axes.flatten()  # Flatten the array of axes for easier iteration

# Loop over each gene and plot on the corresponding axes
for idx, gene in enumerate(protein.var_names):
    sc.pl.umap(protein, color=gene, title=f'UMAP colored by {gene}', ax=axes[idx], show=False, frameon=False, cmap='RdPu')

# Hide any unused axes if the number of genes is not a perfect multiple of ncols
for ax in axes[idx+1:]:
    ax.set_visible(False)

plt.tight_layout()